In [1]:
import pandas as pd
from pybatfish import Session
from pybatfish.datamodel import *

pd.set_option("display.width", 300) 
pd.set_option("display.max_columns", 20) 
pd.set_option("display.max_rows", 1000) 
pd.set_option("display.max_colwidth", None)

# Configure all pybatfish loggers to use WARN level
import logging
logging.getLogger('pybatfish').setLevel(logging.WARN)

In [2]:
bf = Session(host="localhost")



#### Packet Forwarding

* [Traceroute](#Traceroute)
* [Bi-directional Traceroute](#Bi-directional-Traceroute)
* [Reachability](#Reachability)
* [Bi-directional Reachability](#Bi-directional-Reachability)
* [Loop detection](#Loop-detection)
* [Multipath Consistency for host-subnets](#Multipath-Consistency-for-host-subnets)
* [Multipath Consistency for router loopbacks](#Multipath-Consistency-for-router-loopbacks)

In [3]:
bf.set_network('generate_questions')

'generate_questions'

In [4]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### Traceroute

Traces the path(s) for the specified flow.

Performs a virtual traceroute in the network from a starting node. A destination IP and ingress (source) node must be specified. Other IP headers are given default values if unspecified.
Unlike a real traceroute, this traceroute is directional. That is, for it to succeed, the reverse connectivity is not needed. This feature can help debug connectivity issues by decoupling the two directions.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
startLocation | Location (node and interface combination) to start tracing from. | [LocationSpec](../specifiers.md#location-specifier) | False | 
headers | Packet header constraints. | [HeaderConstraints](../datamodel.rst#pybatfish.datamodel.flow.HeaderConstraints) | False | 
maxTraces | Limit the number of traces returned. | int | True | 
ignoreFilters | If set, filters/ACLs encountered along the path are ignored. | bool | True | 

###### Invocation

In [5]:
result = bf.q.traceroute(startLocation='@enter(as2border1[GigabitEthernet2/0])', headers=HeaderConstraints(dstIps='2.34.201.10', srcIps='8.8.8.8')).answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Flow | The flow | [Flow](../datamodel.rst#pybatfish.datamodel.flow.Flow)
Traces | The traces for this flow | Set of [Trace](../datamodel.rst#pybatfish.datamodel.flow.Trace)
TraceCount | The total number traces for this flow | int

Retrieving the flow definition

In [6]:
result.Flow

0    start=as2border1 interface=GigabitEthernet2/0 [8.8.8.8:49152->2.34.201.10:33434 UDP length=512]
Name: Flow, dtype: object

Retrieving the detailed Trace information

In [7]:
len(result.Traces)

1

In [8]:
result.Traces[0]

ListWrapper([((RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.12.11.2, Output Interface: GigabitEthernet1/0, Routes: [ospfE2 (Network: 2.34.201.0/24, Next Hop IP:2.12.11.2)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.23.12.3, Output Interface: GigabitEthernet3/0, Routes: [ospfE2 (Network: 2.34.201.0/24, Next Hop IP:2.23.12.3)]), TRANSMITTED(GigabitEthernet3/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.34.201.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet2/0), DELIVERED_TO_SUBNET(Output Interface: GigabitEthernet2/0, Resolved Next Hop IP: 2.34.201.10))), ((RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.12.12.2, Output Interface: GigabitEthernet2/0, Routes: [ospfE2 (Network: 2.34.201.0/24, Next Hop IP:2.12.12.2)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: 2.23.22.3, Output Interface: GigabitEthernet2/0, Routes: [ospfE2 (Network: 2.34.201.0/24, Next Hop IP:2.23.22.3)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.34.201.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet2/0), DELIVERED_TO_SUBNET(Output Interface: GigabitEthernet2/0, Resolved Next Hop IP: 2.34.201.10)))])

Evaluating the first Trace

In [9]:
result.Traces[0][0]

Trace(disposition='DELIVERED_TO_SUBNET', hops=[Hop(node='as2border1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 110, 'metric': 20, 'network': '2.34.201.0/24', 'nextHopIp': '2.12.11.2', 'protocol': 'ospfE2'}], arpIp='2.12.11.2', outputInterface='GigabitEthernet1/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2core1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 110, 'metric': 20, 'network': '2.34.201.0/24', 'nextHopIp': '2.23.12.3', 'protocol': 'ospfE2'}], arpIp='2.23.12.3', outputInterface='GigabitEthernet3/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet3/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dist2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 0, 'metric': 0, 'network': '2.34.201.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}], arpIp='AUTO/NONE(-1l)', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED'), Step(detail=DeliveredStepDetail(outputInterface='GigabitEthernet2/0', resolvedNexthopIp='2.34.201.10'), action='DELIVERED_TO_SUBNET')])])

Retrieving the disposition of the first Trace

In [10]:
result.Traces[0][0].disposition

'DELIVERED_TO_SUBNET'

Retrieving the first hop of the first Trace

In [11]:
result.Traces[0][0][0]

Hop(node='as2border1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 110, 'metric': 20, 'network': '2.34.201.0/24', 'nextHopIp': '2.12.11.2', 'protocol': 'ospfE2'}], arpIp='2.12.11.2', outputInterface='GigabitEthernet1/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', transformedFlow=None), action='TRANSMITTED')])

Retrieving the last hop of the first Trace

In [12]:
result.Traces[0][0][-1]

Hop(node='as2dist2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 0, 'metric': 0, 'network': '2.34.201.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}], arpIp='AUTO/NONE(-1l)', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED'), Step(detail=DeliveredStepDetail(outputInterface='GigabitEthernet2/0', resolvedNexthopIp='2.34.201.10'), action='DELIVERED_TO_SUBNET')])

In [13]:
bf.set_network('generate_questions')

'generate_questions'

In [14]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### Bi-directional Traceroute

Traces the path(s) for the specified flow, along with path(s) for reverse flows.

This question performs a virtual traceroute in the network from a starting node. A destination IP and ingress (source) node must be specified. Other IP headers are given default values if unspecified.
If the trace succeeds, a traceroute is performed in the reverse direction.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
startLocation | Location (node and interface combination) to start tracing from. | [LocationSpec](../specifiers.md#location-specifier) | False | 
headers | Packet header constraints. | [HeaderConstraints](../datamodel.rst#pybatfish.datamodel.flow.HeaderConstraints) | False | 
maxTraces | Limit the number of traces returned. | int | True | 
ignoreFilters | If set, filters/ACLs encountered along the path are ignored. | bool | True | 

###### Invocation

In [15]:
result = bf.q.bidirectionalTraceroute(startLocation='@enter(as2border1[GigabitEthernet2/0])', headers=HeaderConstraints(dstIps='2.34.201.10', srcIps='8.8.8.8')).answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Forward_Flow | The forward flow. | [Flow](../datamodel.rst#pybatfish.datamodel.flow.Flow)
Forward_Traces | The forward traces. | List of [Trace](../datamodel.rst#pybatfish.datamodel.flow.Trace)
New_Sessions | Sessions initialized by the forward trace. | List of str
Reverse_Flow | The reverse flow. | [Flow](../datamodel.rst#pybatfish.datamodel.flow.Flow)
Reverse_Traces | The reverse traces. | List of [Trace](../datamodel.rst#pybatfish.datamodel.flow.Trace)

Retrieving the Forward flow definition

In [16]:
result.Forward_Flow

0    start=as2border1 interface=GigabitEthernet2/0 [8.8.8.8:49152->2.34.201.10:33434 UDP length=512]
Name: Forward_Flow, dtype: object

Retrieving the detailed Forward Trace information

In [17]:
len(result.Forward_Traces)

1

In [18]:
result.Forward_Traces[0]

ListWrapper([((RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.12.11.2, Output Interface: GigabitEthernet1/0, Routes: [ospfE2 (Network: 2.34.201.0/24, Next Hop IP:2.12.11.2)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.23.12.3, Output Interface: GigabitEthernet3/0, Routes: [ospfE2 (Network: 2.34.201.0/24, Next Hop IP:2.23.12.3)]), TRANSMITTED(GigabitEthernet3/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.34.201.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet2/0), DELIVERED_TO_SUBNET(Output Interface: GigabitEthernet2/0, Resolved Next Hop IP: 2.34.201.10))), ((RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.12.12.2, Output Interface: GigabitEthernet2/0, Routes: [ospfE2 (Network: 2.34.201.0/24, Next Hop IP:2.12.12.2)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: 2.23.22.3, Output Interface: GigabitEthernet2/0, Routes: [ospfE2 (Network: 2.34.201.0/24, Next Hop IP:2.23.22.3)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.34.201.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet2/0), DELIVERED_TO_SUBNET(Output Interface: GigabitEthernet2/0, Resolved Next Hop IP: 2.34.201.10)))])

Evaluating the first Forward Trace

In [19]:
result.Forward_Traces[0][0]

Trace(disposition='DELIVERED_TO_SUBNET', hops=[Hop(node='as2border1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 110, 'metric': 20, 'network': '2.34.201.0/24', 'nextHopIp': '2.12.11.2', 'protocol': 'ospfE2'}], arpIp='2.12.11.2', outputInterface='GigabitEthernet1/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2core1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 110, 'metric': 20, 'network': '2.34.201.0/24', 'nextHopIp': '2.23.12.3', 'protocol': 'ospfE2'}], arpIp='2.23.12.3', outputInterface='GigabitEthernet3/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet3/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dist2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 0, 'metric': 0, 'network': '2.34.201.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}], arpIp='AUTO/NONE(-1l)', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED'), Step(detail=DeliveredStepDetail(outputInterface='GigabitEthernet2/0', resolvedNexthopIp='2.34.201.10'), action='DELIVERED_TO_SUBNET')])])

Retrieving the disposition of the first Forward Trace

In [20]:
result.Forward_Traces[0][0].disposition

'DELIVERED_TO_SUBNET'

Retrieving the first hop of the first Forward Trace

In [21]:
result.Forward_Traces[0][0][0]

Hop(node='as2border1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 110, 'metric': 20, 'network': '2.34.201.0/24', 'nextHopIp': '2.12.11.2', 'protocol': 'ospfE2'}], arpIp='2.12.11.2', outputInterface='GigabitEthernet1/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', transformedFlow=None), action='TRANSMITTED')])

Retrieving the last hop of the first Forward Trace

In [22]:
result.Forward_Traces[0][0][-1]

Hop(node='as2dist2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 0, 'metric': 0, 'network': '2.34.201.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}], arpIp='AUTO/NONE(-1l)', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED'), Step(detail=DeliveredStepDetail(outputInterface='GigabitEthernet2/0', resolvedNexthopIp='2.34.201.10'), action='DELIVERED_TO_SUBNET')])

Retrieving the Return flow definition

In [23]:
result.Reverse_Flow

0    start=as2dist2 interface=GigabitEthernet2/0 [2.34.201.10:33434->8.8.8.8:49152 UDP length=512]
Name: Reverse_Flow, dtype: object

Retrieving the detailed Return Trace information

In [24]:
len(result.Reverse_Traces)

1

In [25]:
result.Reverse_Traces[0]

ListWrapper([((RECEIVED(GigabitEthernet2/0), NO_ROUTE))])

Evaluating the first Reverse Trace

In [26]:
result.Reverse_Traces[0][0]

Trace(disposition='NO_ROUTE', hops=[Hop(node='as2dist2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[], arpIp=None, outputInterface=None), action='NO_ROUTE')])])

Retrieving the disposition of the first Reverse Trace

In [27]:
result.Reverse_Traces[0][0].disposition

'NO_ROUTE'

Retrieving the first hop of the first Reverse Trace

In [28]:
result.Reverse_Traces[0][0][0]

Hop(node='as2dist2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[], arpIp=None, outputInterface=None), action='NO_ROUTE')])

Retrieving the last hop of the first Reverse Trace

In [29]:
result.Reverse_Traces[0][0][-1]

Hop(node='as2dist2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[], arpIp=None, outputInterface=None), action='NO_ROUTE')])

In [30]:
bf.set_network('generate_questions')

'generate_questions'

In [31]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### Reachability

Finds flows that match the specified path and header space conditions.

Searches across all flows that match the specified conditions and returns examples of such flows. This question can be used to ensure that certain services are globally accessible and parts of the network are perfectly isolated from each other.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
pathConstraints | Constraint the path a flow can take (start/end/transit locations). | [PathConstraints](../datamodel.rst#pybatfish.datamodel.flow.PathConstraints) | True | 
headers | Packet header constraints. | [HeaderConstraints](../datamodel.rst#pybatfish.datamodel.flow.HeaderConstraints) | True | 
actions | Only return flows for which the disposition is from this set. | [DispositionSpec](../specifiers.md#disposition-specifier) | True | success
maxTraces | Limit the number of traces returned. | int | True | 
invertSearch | Search for packet headers outside the specified headerspace, rather than inside the space. | bool | True | 
ignoreFilters | Do not apply filters/ACLs during analysis. | bool | True | 

###### Invocation

In [32]:
result = bf.q.reachability(pathConstraints=PathConstraints(startLocation = '/as2/'), headers=HeaderConstraints(dstIps='host1', srcIps='0.0.0.0/0', applications='DNS'), actions='SUCCESS').answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Flow | The flow | [Flow](../datamodel.rst#pybatfish.datamodel.flow.Flow)
Traces | The traces for this flow | Set of [Trace](../datamodel.rst#pybatfish.datamodel.flow.Trace)
TraceCount | The total number traces for this flow | int

Retrieving the flow definition

In [33]:
result.Flow

0    start=as2border1 [10.0.0.0:49152->2.128.0.101:53 UDP length=512]
1    start=as2border2 [10.0.0.0:49152->2.128.0.101:53 UDP length=512]
2      start=as2core1 [10.0.0.0:49152->2.128.0.101:53 UDP length=512]
3      start=as2core2 [10.0.0.0:49152->2.128.0.101:53 UDP length=512]
4      start=as2dept1 [10.0.0.0:49152->2.128.0.101:53 UDP length=512]
5      start=as2dist1 [10.0.0.0:49152->2.128.0.101:53 UDP length=512]
6      start=as2dist2 [10.0.0.0:49152->2.128.0.101:53 UDP length=512]
Name: Flow, dtype: object

Retrieving the detailed Trace information

In [34]:
len(result.Traces)

7

In [35]:
result.Traces[0]

ListWrapper([((ORIGINATED(default), FORWARDED(ARP IP: 2.12.11.2, Output Interface: GigabitEthernet1/0, Routes: [ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.101.4),ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.201.4)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.23.11.3, Output Interface: GigabitEthernet2/0, Routes: [ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.101.4)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.34.101.4, Output Interface: GigabitEthernet2/0, Routes: [bgp (Network: 2.128.0.0/24, Next Hop IP:2.34.101.4)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(eth0), PERMITTED(filter::INPUT (INGRESS_FILTER)), ACCEPTED(eth0))), ((ORIGINATED(default), FORWARDED(ARP IP: 2.12.11.2, Output Interface: GigabitEthernet1/0, Routes: [ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.101.4),ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.201.4)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.23.12.3, Output Interface: GigabitEthernet3/0, Routes: [ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.201.4)]), TRANSMITTED(GigabitEthernet3/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: 2.34.201.4, Output Interface: GigabitEthernet2/0, Routes: [bgp (Network: 2.128.0.0/24, Next Hop IP:2.34.201.4)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(eth0), PERMITTED(filter::INPUT (INGRESS_FILTER)), ACCEPTED(eth0))), ((ORIGINATED(default), FORWARDED(ARP IP: 2.12.12.2, Output Interface: GigabitEthernet2/0, Routes: [ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.101.4),ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.201.4)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: 2.23.22.3, Output Interface: GigabitEthernet2/0, Routes: [ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.201.4)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.34.201.4, Output Interface: GigabitEthernet2/0, Routes: [bgp (Network: 2.128.0.0/24, Next Hop IP:2.34.201.4)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(eth0), PERMITTED(filter::INPUT (INGRESS_FILTER)), ACCEPTED(eth0))), ((ORIGINATED(default), FORWARDED(ARP IP: 2.12.12.2, Output Interface: GigabitEthernet2/0, Routes: [ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.101.4),ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.201.4)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: 2.23.21.3, Output Interface: GigabitEthernet3/0, Routes: [ibgp (Network: 2.128.0.0/24, Next Hop IP:2.34.101.4)]), TRANSMITTED(GigabitEthernet3/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: 2.34.101.4, Output Interface: GigabitEthernet2/0, Routes: [bgp (Network: 2.128.0.0/24, Next Hop IP:2.34.101.4)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(eth0), PERMITTED(filter::INPUT (INGRESS_FILTER)), ACCEPTED(eth0)))])

Evaluating the first Trace

In [36]:
result.Traces[0][0]

Trace(disposition='ACCEPTED', hops=[Hop(node='as2border1', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'admin': 200, 'metric': 50, 'network': '2.128.0.0/24', 'nextHopIp': '2.34.101.4', 'protocol': 'ibgp'}, {'admin': 200, 'metric': 50, 'network': '2.128.0.0/24', 'nextHopIp': '2.34.201.4', 'protocol': 'ibgp'}], arpIp='2.12.11.2', outputInterface='GigabitEthernet1/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2core1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 200, 'metric': 50, 'network': '2.128.0.0/24', 'nextHopIp': '2.34.101.4', 'protocol': 'ibgp'}], arpIp='2.23.11.3', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dist1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 20, 'metric': 50, 'network': '2.128.0.0/24', 'nextHopIp': '2.34.101.4', 'protocol': 'bgp'}], arpIp='2.34.101.4', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dept1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 0, 'metric': 0, 'network': '2.128.0.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}], arpIp='AUTO/NONE(-1l)', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='host1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='eth0', inputVrf='default'), action='RECEIVED'), Step(detail=FilterStepDetail(filter='filter::INPUT', filterType='INGRESS_FILTER', inputInterface='eth0', flow=Flow(dscp=0, dstIp='2.128.0.101', dstPort=53, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='as2border1', ingressVrf='default', ipProtocol='UDP', packetLength=512, srcIp='10.0.0.0', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=0, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=InboundStepDetail(interface='eth0'), action='ACCEPTED')])])

Retrieving the disposition of the first Trace

In [37]:
result.Traces[0][0].disposition

'ACCEPTED'

Retrieving the first hop of the first Trace

In [38]:
result.Traces[0][0][0]

Hop(node='as2border1', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'admin': 200, 'metric': 50, 'network': '2.128.0.0/24', 'nextHopIp': '2.34.101.4', 'protocol': 'ibgp'}, {'admin': 200, 'metric': 50, 'network': '2.128.0.0/24', 'nextHopIp': '2.34.201.4', 'protocol': 'ibgp'}], arpIp='2.12.11.2', outputInterface='GigabitEthernet1/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', transformedFlow=None), action='TRANSMITTED')])

Retrieving the last hop of the first Trace

In [39]:
result.Traces[0][0][-1]

Hop(node='host1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='eth0', inputVrf='default'), action='RECEIVED'), Step(detail=FilterStepDetail(filter='filter::INPUT', filterType='INGRESS_FILTER', inputInterface='eth0', flow=Flow(dscp=0, dstIp='2.128.0.101', dstPort=53, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='as2border1', ingressVrf='default', ipProtocol='UDP', packetLength=512, srcIp='10.0.0.0', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=0, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=InboundStepDetail(interface='eth0'), action='ACCEPTED')])

In [40]:
bf.set_network('generate_questions')

'generate_questions'

In [41]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### Bi-directional Reachability

Searches for successfully delivered flows that can successfully receive a response.

Performs two reachability analyses, first originating from specified sources, then returning back to those sources. After the first (forward) pass, sets up sessions in the network and creates returning flows for each successfully delivered forward flow. The second pass searches for return flows that can be successfully delivered in the presence of the setup sessions.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
pathConstraints | Constraint the path a flow can take (start/end/transit locations). | [PathConstraints](../datamodel.rst#pybatfish.datamodel.flow.PathConstraints) | True | 
headers | Packet header constraints. | [HeaderConstraints](../datamodel.rst#pybatfish.datamodel.flow.HeaderConstraints) | False | 
returnFlowType | Specifies the type of return flows to search. | str | True | SUCCESS

###### Invocation

In [42]:
result = bf.q.bidirectionalReachability(pathConstraints=PathConstraints(startLocation = '/as2dist1/'), headers=HeaderConstraints(dstIps='host1', srcIps='0.0.0.0/0', applications='DNS'), returnFlowType='SUCCESS').answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Forward_Flow | The forward flow. | [Flow](../datamodel.rst#pybatfish.datamodel.flow.Flow)
Forward_Traces | The forward traces. | List of [Trace](../datamodel.rst#pybatfish.datamodel.flow.Trace)
New_Sessions | Sessions initialized by the forward trace. | List of str
Reverse_Flow | The reverse flow. | [Flow](../datamodel.rst#pybatfish.datamodel.flow.Flow)
Reverse_Traces | The reverse traces. | List of [Trace](../datamodel.rst#pybatfish.datamodel.flow.Trace)

Retrieving the Forward flow definition

In [43]:
result.Forward_Flow

0    start=as2dist1 [2.34.101.3:49152->2.128.0.101:53 UDP length=512]
Name: Forward_Flow, dtype: object

Retrieving the detailed Forward Trace information

In [44]:
len(result.Forward_Traces)

1

In [45]:
result.Forward_Traces[0]

ListWrapper([((ORIGINATED(default), FORWARDED(ARP IP: 2.34.101.4, Output Interface: GigabitEthernet2/0, Routes: [bgp (Network: 2.128.0.0/24, Next Hop IP:2.34.101.4)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet2/0, Routes: [connected (Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(eth0), PERMITTED(filter::INPUT (INGRESS_FILTER)), ACCEPTED(eth0)))])

Evaluating the first Forward Trace

In [46]:
result.Forward_Traces[0][0]

Trace(disposition='ACCEPTED', hops=[Hop(node='as2dist1', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'admin': 20, 'metric': 50, 'network': '2.128.0.0/24', 'nextHopIp': '2.34.101.4', 'protocol': 'bgp'}], arpIp='2.34.101.4', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dept1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 0, 'metric': 0, 'network': '2.128.0.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}], arpIp='AUTO/NONE(-1l)', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='host1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='eth0', inputVrf='default'), action='RECEIVED'), Step(detail=FilterStepDetail(filter='filter::INPUT', filterType='INGRESS_FILTER', inputInterface='eth0', flow=Flow(dscp=0, dstIp='2.128.0.101', dstPort=53, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='as2dist1', ingressVrf='default', ipProtocol='UDP', packetLength=512, srcIp='2.34.101.3', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=0, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=InboundStepDetail(interface='eth0'), action='ACCEPTED')])])

Retrieving the disposition of the first Forward Trace

In [47]:
result.Forward_Traces[0][0].disposition

'ACCEPTED'

Retrieving the first hop of the first Forward Trace

In [48]:
result.Forward_Traces[0][0][0]

Hop(node='as2dist1', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'admin': 20, 'metric': 50, 'network': '2.128.0.0/24', 'nextHopIp': '2.34.101.4', 'protocol': 'bgp'}], arpIp='2.34.101.4', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED')])

Retrieving the last hop of the first Forward Trace

In [49]:
result.Forward_Traces[0][0][-1]

Hop(node='host1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='eth0', inputVrf='default'), action='RECEIVED'), Step(detail=FilterStepDetail(filter='filter::INPUT', filterType='INGRESS_FILTER', inputInterface='eth0', flow=Flow(dscp=0, dstIp='2.128.0.101', dstPort=53, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='as2dist1', ingressVrf='default', ipProtocol='UDP', packetLength=512, srcIp='2.34.101.3', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=0, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=InboundStepDetail(interface='eth0'), action='ACCEPTED')])

Retrieving the Return flow definition

In [50]:
result.Reverse_Flow

0    start=host1 [2.128.0.101:53->2.34.101.3:49152 UDP length=512]
Name: Reverse_Flow, dtype: object

Retrieving the detailed Return Trace information

In [51]:
len(result.Reverse_Traces)

1

In [52]:
result.Reverse_Traces[0]

ListWrapper([((ORIGINATED(default), FORWARDED(ARP IP: 2.128.0.1, Output Interface: eth0, Routes: [static (Network: 0.0.0.0/0, Next Hop IP:2.128.0.1)]), PERMITTED(filter::OUTPUT (EGRESS_FILTER)), TRANSMITTED(eth0)), (RECEIVED(GigabitEthernet2/0), PERMITTED(RESTRICT_HOST_TRAFFIC_IN (INGRESS_FILTER)), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet0/0, Routes: [connected (Network: 2.34.101.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet0/0)), (RECEIVED(GigabitEthernet2/0), ACCEPTED(GigabitEthernet2/0)))])

Evaluating the first Reverse Trace

In [53]:
result.Reverse_Traces[0][0]

Trace(disposition='ACCEPTED', hops=[Hop(node='host1', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'admin': 1, 'metric': 0, 'network': '0.0.0.0/0', 'nextHopIp': '2.128.0.1', 'protocol': 'static'}], arpIp='2.128.0.1', outputInterface='eth0'), action='FORWARDED'), Step(detail=FilterStepDetail(filter='filter::OUTPUT', filterType='EGRESS_FILTER', inputInterface='', flow=Flow(dscp=0, dstIp='2.34.101.3', dstPort=49152, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='host1', ingressVrf='default', ipProtocol='UDP', packetLength=512, srcIp='2.128.0.101', srcPort=53, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=0, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='eth0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dept1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=FilterStepDetail(filter='RESTRICT_HOST_TRAFFIC_IN', filterType='INGRESS_FILTER', inputInterface='GigabitEthernet2/0', flow=Flow(dscp=0, dstIp='2.34.101.3', dstPort=49152, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='host1', ingressVrf='default', ipProtocol='UDP', packetLength=512, srcIp='2.128.0.101', srcPort=53, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=0, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=RoutingStepDetail(routes=[{'admin': 0, 'metric': 0, 'network': '2.34.101.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}], arpIp='AUTO/NONE(-1l)', outputInterface='GigabitEthernet0/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet0/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dist1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=InboundStepDetail(interface='GigabitEthernet2/0'), action='ACCEPTED')])])

Retrieving the disposition of the first Reverse Trace

In [54]:
result.Reverse_Traces[0][0].disposition

'ACCEPTED'

Retrieving the first hop of the first Reverse Trace

In [55]:
result.Reverse_Traces[0][0][0]

Hop(node='host1', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'admin': 1, 'metric': 0, 'network': '0.0.0.0/0', 'nextHopIp': '2.128.0.1', 'protocol': 'static'}], arpIp='2.128.0.1', outputInterface='eth0'), action='FORWARDED'), Step(detail=FilterStepDetail(filter='filter::OUTPUT', filterType='EGRESS_FILTER', inputInterface='', flow=Flow(dscp=0, dstIp='2.34.101.3', dstPort=49152, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface=None, ingressNode='host1', ingressVrf='default', ipProtocol='UDP', packetLength=512, srcIp='2.128.0.101', srcPort=53, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=0, tcpFlagsUrg=0)), action='PERMITTED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='eth0', transformedFlow=None), action='TRANSMITTED')])

Retrieving the last hop of the first Reverse Trace

In [56]:
result.Reverse_Traces[0][0][-1]

Hop(node='as2dist1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=InboundStepDetail(interface='GigabitEthernet2/0'), action='ACCEPTED')])

In [57]:
bf.set_network('generate_questions')

'generate_questions'

In [58]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### Loop detection

Detects forwarding loops.

Searches across all possible flows in the network and returns example flows that will experience forwarding loops.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
maxTraces | Limit the number of traces returned. | int | True | 

###### Invocation

In [59]:
result = bf.q.detectLoops().answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Flow | The flow | [Flow](../datamodel.rst#pybatfish.datamodel.flow.Flow)
Traces | The traces for this flow | Set of [Trace](../datamodel.rst#pybatfish.datamodel.flow.Trace)
TraceCount | The total number traces for this flow | int

Print the first 5 rows of the returned Dataframe

In [60]:
result.head(5)

,Flow,Traces,TraceCount


In [61]:
bf.set_network('generate_questions')

'generate_questions'

In [62]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### Multipath Consistency for host-subnets

Validates multipath consistency between all pairs of subnets.

Searches across all flows between subnets that are treated differently (i.e., dropped versus forwarded) by different paths in the network and returns example flows.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
maxTraces | Limit the number of traces returned. | int | True | 

###### Invocation

In [63]:
result = bf.q.subnetMultipathConsistency().answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Flow | The flow | [Flow](../datamodel.rst#pybatfish.datamodel.flow.Flow)
Traces | The traces for this flow | Set of [Trace](../datamodel.rst#pybatfish.datamodel.flow.Trace)
TraceCount | The total number traces for this flow | int

Retrieving the flow definition

In [64]:
result.Flow

0    start=as2dept1 interface=GigabitEthernet0/0 [2.34.101.1:49152->1.0.1.3:23 TCP length=512]
1    start=as2dept1 interface=GigabitEthernet1/0 [2.34.201.1:49152->1.0.1.3:23 TCP length=512]
2     start=as2dept1 interface=GigabitEthernet2/0 [2.128.0.2:49152->1.0.1.3:23 TCP length=512]
3     start=as2dept1 interface=GigabitEthernet3/0 [2.128.1.2:49152->1.0.1.3:23 TCP length=512]
4     start=as2dist1 interface=GigabitEthernet0/0 [2.23.11.1:49152->1.0.1.3:23 TCP length=512]
5     start=as2dist1 interface=GigabitEthernet1/0 [2.23.21.1:49152->1.0.1.3:23 TCP length=512]
6    start=as2dist1 interface=GigabitEthernet2/0 [2.34.101.1:49152->1.0.1.3:23 TCP length=512]
7     start=as2dist2 interface=GigabitEthernet0/0 [2.23.22.1:49152->1.0.1.3:23 TCP length=512]
8     start=as2dist2 interface=GigabitEthernet1/0 [2.23.12.1:49152->1.0.1.3:23 TCP length=512]
9    start=as2dist2 interface=GigabitEthernet2/0 [2.34.201.1:49152->1.0.1.3:23 TCP length=512]
Name: Flow, dtype: object

Retrieving the detailed Trace information

In [65]:
len(result.Traces)

10

In [66]:
result.Traces[0]

ListWrapper([((RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.34.101.3, Output Interface: GigabitEthernet0/0, Routes: [bgp (Network: 1.0.1.0/24, Next Hop IP:2.34.101.3)]), TRANSMITTED(GigabitEthernet0/0)), (RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.23.11.2, Output Interface: GigabitEthernet0/0, Routes: [ibgp (Network: 1.0.1.0/24, Next Hop IP:10.12.11.1)]), TRANSMITTED(GigabitEthernet0/0)), (RECEIVED(GigabitEthernet2/0), DENIED(blocktelnet (INGRESS_FILTER)))), ((RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.34.101.3, Output Interface: GigabitEthernet0/0, Routes: [bgp (Network: 1.0.1.0/24, Next Hop IP:2.34.101.3)]), TRANSMITTED(GigabitEthernet0/0)), (RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.23.21.2, Output Interface: GigabitEthernet1/0, Routes: [ibgp (Network: 1.0.1.0/24, Next Hop IP:10.12.11.1)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet3/0), FORWARDED(ARP IP: 2.12.12.1, Output Interface: GigabitEthernet1/0, Routes: [ibgp (Network: 1.0.1.0/24, Next Hop IP:10.12.11.1)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 10.12.11.1, Output Interface: GigabitEthernet0/0, Routes: [bgp (Network: 1.0.1.0/24, Next Hop IP:10.12.11.1)]), PERMITTED(INSIDE_TO_AS1 (EGRESS_FILTER)), TRANSMITTED(GigabitEthernet0/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet0/0, Routes: [connected (Network: 1.0.1.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet0/0), DELIVERED_TO_SUBNET(Output Interface: GigabitEthernet0/0, Resolved Next Hop IP: 1.0.1.3))), ((RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.34.201.3, Output Interface: GigabitEthernet1/0, Routes: [bgp (Network: 1.0.1.0/24, Next Hop IP:2.34.201.3)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.23.22.2, Output Interface: GigabitEthernet0/0, Routes: [ibgp (Network: 1.0.1.0/24, Next Hop IP:10.12.11.1)]), TRANSMITTED(GigabitEthernet0/0)), (RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.12.12.1, Output Interface: GigabitEthernet1/0, Routes: [ibgp (Network: 1.0.1.0/24, Next Hop IP:10.12.11.1)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 10.12.11.1, Output Interface: GigabitEthernet0/0, Routes: [bgp (Network: 1.0.1.0/24, Next Hop IP:10.12.11.1)]), PERMITTED(INSIDE_TO_AS1 (EGRESS_FILTER)), TRANSMITTED(GigabitEthernet0/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: AUTO/NONE(-1l), Output Interface: GigabitEthernet0/0, Routes: [connected (Network: 1.0.1.0/24, Next Hop IP:AUTO/NONE(-1l))]), TRANSMITTED(GigabitEthernet0/0), DELIVERED_TO_SUBNET(Output Interface: GigabitEthernet0/0, Resolved Next Hop IP: 1.0.1.3))), ((RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.34.201.3, Output Interface: GigabitEthernet1/0, Routes: [bgp (Network: 1.0.1.0/24, Next Hop IP:2.34.201.3)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.23.12.2, Output Interface: GigabitEthernet1/0, Routes: [ibgp (Network: 1.0.1.0/24, Next Hop IP:10.12.11.1)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet3/0), DENIED(blocktelnet (INGRESS_FILTER))))])

Evaluating the first Trace

In [67]:
result.Traces[0][0]

Trace(disposition='DENIED_IN', hops=[Hop(node='as2dept1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 20, 'metric': 50, 'network': '1.0.1.0/24', 'nextHopIp': '2.34.101.3', 'protocol': 'bgp'}], arpIp='2.34.101.3', outputInterface='GigabitEthernet0/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet0/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dist1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 200, 'metric': 50, 'network': '1.0.1.0/24', 'nextHopIp': '10.12.11.1', 'protocol': 'ibgp'}], arpIp='2.23.11.2', outputInterface='GigabitEthernet0/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet0/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2core1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=FilterStepDetail(filter='blocktelnet', filterType='INGRESS_FILTER', inputInterface='GigabitEthernet2/0', flow=Flow(dscp=0, dstIp='1.0.1.3', dstPort=23, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface='GigabitEthernet0/0', ingressNode='as2dept1', ingressVrf=None, ipProtocol='TCP', packetLength=512, srcIp='2.34.101.1', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=1, tcpFlagsUrg=0)), action='DENIED')])])

Retrieving the disposition of the first Trace

In [68]:
result.Traces[0][0].disposition

'DENIED_IN'

Retrieving the first hop of the first Trace

In [69]:
result.Traces[0][0][0]

Hop(node='as2dept1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 20, 'metric': 50, 'network': '1.0.1.0/24', 'nextHopIp': '2.34.101.3', 'protocol': 'bgp'}], arpIp='2.34.101.3', outputInterface='GigabitEthernet0/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet0/0', transformedFlow=None), action='TRANSMITTED')])

Retrieving the last hop of the first Trace

In [70]:
result.Traces[0][0][-1]

Hop(node='as2core1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet2/0', inputVrf='default'), action='RECEIVED'), Step(detail=FilterStepDetail(filter='blocktelnet', filterType='INGRESS_FILTER', inputInterface='GigabitEthernet2/0', flow=Flow(dscp=0, dstIp='1.0.1.3', dstPort=23, ecn=0, fragmentOffset=0, icmpCode=None, icmpVar=None, ingressInterface='GigabitEthernet0/0', ingressNode='as2dept1', ingressVrf=None, ipProtocol='TCP', packetLength=512, srcIp='2.34.101.1', srcPort=49152, tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=1, tcpFlagsUrg=0)), action='DENIED')])

In [71]:
bf.set_network('generate_questions')

'generate_questions'

In [72]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### Multipath Consistency for router loopbacks

Validates multipath consistency between all pairs of loopbacks.

Finds flows between loopbacks that are treated differently (i.e., dropped versus forwarded) by different paths in the presence of multipath routing.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
maxTraces | Limit the number of traces returned. | int | True | 

###### Invocation

In [73]:
result = bf.q.loopbackMultipathConsistency().answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Flow | The flow | [Flow](../datamodel.rst#pybatfish.datamodel.flow.Flow)
Traces | The traces for this flow | Set of [Trace](../datamodel.rst#pybatfish.datamodel.flow.Trace)
TraceCount | The total number traces for this flow | int

Retrieving the flow definition

In [74]:
result.Flow

0    start=as2core2 [2.1.2.2:49152->2.1.2.1:23 TCP length=512]
1    start=as2dist1 [2.1.3.1:49152->2.1.1.1:23 TCP length=512]
2    start=as2dist2 [2.1.3.2:49152->2.1.1.1:23 TCP length=512]
Name: Flow, dtype: object

Retrieving the detailed Trace information

In [75]:
len(result.Traces)

3

In [76]:
result.Traces[0]

ListWrapper([((ORIGINATED(default), FORWARDED(ARP IP: 2.12.22.1, Output Interface: GigabitEthernet0/0, Routes: [ospf (Network: 2.1.2.1/32, Next Hop IP:2.12.22.1)]), TRANSMITTED(GigabitEthernet0/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: 2.12.21.2, Output Interface: GigabitEthernet2/0, Routes: [ospf (Network: 2.1.2.1/32, Next Hop IP:2.12.21.2)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet1/0), ACCEPTED(Loopback0))), ((ORIGINATED(default), FORWARDED(ARP IP: 2.12.12.1, Output Interface: GigabitEthernet1/0, Routes: [ospf (Network: 2.1.2.1/32, Next Hop IP:2.12.12.1)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet2/0), FORWARDED(ARP IP: 2.12.11.2, Output Interface: GigabitEthernet1/0, Routes: [ospf (Network: 2.1.2.1/32, Next Hop IP:2.12.11.2)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet0/0), ACCEPTED(Loopback0))), ((ORIGINATED(default), FORWARDED(ARP IP: 2.23.22.3, Output Interface: GigabitEthernet2/0, Routes: [ospf (Network: 2.1.2.1/32, Next Hop IP:2.23.22.3)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(ARP IP: 2.23.12.2, Output Interface: GigabitEthernet1/0, Routes: [ospf (Network: 2.1.2.1/32, Next Hop IP:2.23.12.2)]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet3/0), DENIED(blocktelnet (INGRESS_FILTER)))), ((ORIGINATED(default), FORWARDED(ARP IP: 2.23.21.3, Output Interface: GigabitEthernet3/0, Routes: [ospf (Network: 2.1.2.1/32, Next Hop IP:2.23.21.3)]), TRANSMITTED(GigabitEthernet3/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(ARP IP: 2.23.11.2, Output Interface: GigabitEthernet0/0, Routes: [ospf (Network: 2.1.2.1/32, Next Hop IP:2.23.11.2)]), TRANSMITTED(GigabitEthernet0/0)), (RECEIVED(GigabitEthernet2/0), DENIED(blocktelnet (INGRESS_FILTER))))])

Evaluating the first Trace

In [77]:
result.Traces[0][0]

Trace(disposition='ACCEPTED', hops=[Hop(node='as2core2', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'admin': 110, 'metric': 3, 'network': '2.1.2.1/32', 'nextHopIp': '2.12.22.1', 'protocol': 'ospf'}], arpIp='2.12.22.1', outputInterface='GigabitEthernet0/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet0/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2border2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1/0', inputVrf='default'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'admin': 110, 'metric': 2, 'network': '2.1.2.1/32', 'nextHopIp': '2.12.21.2', 'protocol': 'ospf'}], arpIp='2.12.21.2', outputInterface='GigabitEthernet2/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2core1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1/0', inputVrf='default'), action='RECEIVED'), Step(detail=InboundStepDetail(interface='Loopback0'), action='ACCEPTED')])])

Retrieving the disposition of the first Trace

In [78]:
result.Traces[0][0].disposition

'ACCEPTED'

Retrieving the first hop of the first Trace

In [79]:
result.Traces[0][0][0]

Hop(node='as2core2', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'admin': 110, 'metric': 3, 'network': '2.1.2.1/32', 'nextHopIp': '2.12.22.1', 'protocol': 'ospf'}], arpIp='2.12.22.1', outputInterface='GigabitEthernet0/0'), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet0/0', transformedFlow=None), action='TRANSMITTED')])

Retrieving the last hop of the first Trace

In [80]:
result.Traces[0][0][-1]

Hop(node='as2core1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1/0', inputVrf='default'), action='RECEIVED'), Step(detail=InboundStepDetail(interface='Loopback0'), action='ACCEPTED')])